In [1]:
import numpy as np
from qibo.models import Circuit
from qibo import gates,models
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import inv
import matplotlib.pylab as plt

# Fermionic FT class

In [2]:
#Class fermionic FT: creates the qibo quantum circuit to perform the fermionic FT. 

#inputs:
    #Nspin --> number of Qbits targets to perform the fermionic FT. It has to be a power of two 2**m
    #nqext--> In the case wants to integrate the fermionic FT in a larger circuit, the number of Qbits of this larger circuit. 
    #         by default is None, the argument is optional only. However, in the case it is specify, one must then add the 
    #         qlistext input.
    #qlistext--> List where the fermionic FT circuit will be add in the large circuit. The len of this list must be equal as Nspin
    #sort: By default is False and then the k states are sort as: 0,n/2 ;1,-(n/2)+1;2,-(n/2)+2;...;n/2-1,-1.
    #      If sort=True then we add additional fermionic SWAP circuit and the final order of the k states is:
    #      0,1,2,3,...,n/2,-n/2+1,-n/2+2,...,-1
    


class fermionicFT:
    def __init__(self,Nspin,nqext=None,qlistext=None,sort=False):
        #obtain the number of qbits we are using
        self.numq=Nspin
        #if the number of qbits is not a power of two or different from 0, rise an error
        if not(self.numq & (self.numq-1) == 0) and self.numq != 0:
            raise ValueError("The number of qbits is not a power of 2")
        
        #create a list with the qbits used in the circuit
        qbit_list=np.arange(0,self.numq)
        
        #compute which power of 2 is it
        self.two_power=int(np.log2(self.numq))
        #call the divideqbit method
        self.divideqbit(qbit_list)
        #call the call the initcirc method
        self.initcirc()
        #call the nextcirc1 method
        self.nextcirc1()
        #call the finalcirc method
        self.finalcirc()
        
        #if sort==True then call the sortFT method
        if sort==True:
            self.sortFT()
        
        #if nqext==None and qlistext==None then the final qc is self.circ0
        if nqext==None and qlistext==None:
            smallqc=self.circ0
            self.fermFTqc=smallqc
        
        #if the user add a qlistext but none the nqext raise and error
        elif nqext==None and not qlistext==None:
            raise ValueError("Specify both, the number of qbits of the large circuit(nqext)and the list of the qbits you want to use (qlistext)")
        
        ##if the user add the nqext but none the qlistext raise and error
        elif not nqext==None and qlistext==None:
            raise ValueError("Specify both, the number of qbits of the large circuit(nqext)and the list of the qbits you want to use (qlistext)")
        
        # if the user specify both, nqext and qlistext, then add the FTcircuit on the specified qbits from a large one circuit 
        else:
            largeqc=Circuit(nqext)
            smallqc=self.circ0
            largeqc.add(smallqc.on_qubits(*qlistext))
            self.fermFTqc=largeqc
            
            
        
        
        
    #divideqbit: divide the input qbit list of n=2**m into lists of 2**i where l goes from 2 to m  
            # qlist -->  input qbit list pass from the user       
    #outputs: 
            # self.divqbit: list with the arrays of the segmented new qbit list. The new output contain first the 2**2 lists  
    def divideqbit(self,qlist):
        #create the self.divqbit as an empty list
        self.divqbit=[]
        #loop to create the new lists with a length of 2**i
        for i in range(2,self.two_power+1):
            #length of the new list
            times=2**i
            #compute the number of lists needed to encode all the qbits for the original list
            miter=int(self.numq/times)
            # loop to save all the original qbits in the new lists of length 2**i
            for j in range(0,miter):
                #save the original qbits in the new list
                newqlist=qlist[times*j:times*(j+1)]
                #append it to the output variable
                self.divqbit.append(newqlist)
    
    
    #initcirc: prepare the initial fSWAP circuit   
            
    #outputs: 
            # self.circ0: Circuit with the fSWAPS, is the first part of the FT circuit
    def initcirc(self):
        #create a circuit with the same number of qbits as the input user qbit list
        self.circ0=Circuit(self.numq)
        #inverse the order of self.divqbit
        newlist=self.divqbit[::-1]
        #explore all the list in newlist
        for i in newlist:
            #create a circuit with the same number of qbits as the input user qbit list
            qclarge=Circuit(self.numq)
            #the number of points used for the FT
            n=len(i)
            #Create the FSWAP circuit corresponding to n qbits
            qcsmall=self.circfSWAP.fSWAPSFT(n)
            #add the small circuit to the large one in the qbits inside our list
            qclarge.add(qcsmall.on_qubits(*i))
            #sum this new circuit to the existent ones
            self.circ0+=qclarge
    
    #nextcirc1: add the 2 qbit fFT in the circuit in all qbits (in this case the 2 qbit FT corresponds to k=0)  
            
    #outputs: 
            # self.circ0: it add the new fFT to the circ0
    def nextcirc1(self):
        #creat a new circuit with the same number of qbits as the input user qbit list
        qc0=Circuit(self.numq)
        #in all the even qbits add the 2-Qbit fermionicFT
        for i in range(0,self.numq,2):
            #add the circuit to the original one
            qc0+=self.circFT.FTkgate(self.numq,i,i+1,0,self.numq)
        #sum the 2-fFT to the self.circ0 circuit
        self.circ0+=qc0
    
    
    #finalcirc: add the rest of the circuit. This corresponds to the additional fermionic SWAPS and the m qbits FT, 
    #           with m from 4 to n
            
    #outputs: 
            # self.circ0: we add the final part to self.circ0
    
    def finalcirc(self):
        #list with the order of self.divqbit
        newlist=self.divqbit
        #explore all the list in newlist
        for i in newlist:
            #create a circuit with the same number of qbits as the input user qbit list
            qclarge=Circuit(self.numq)
            #the number of points used for the FT
            n=len(i)
            #Create the FSWAP circuit corresponding to n qbits
            fSWAPsmall=self.circfSWAP.fSWAPSFT(n)
            #create the inverse swap qbirs
            fSWAPsmallinv=fSWAPsmall.invert()
            #create the small FT            
            FTksmall=self.circFT.joinFT(n)
            #if we are in the last step not add the fSWAPsmall circuit 
            if len(i)==self.numq:
                qcfinsmall=fSWAPsmallinv+FTksmall
            else:
                qcfinsmall=fSWAPsmallinv+FTksmall+fSWAPsmall
            #add the small circuit to the large one in the qbits inside our list
            qclarge.add(qcfinsmall.on_qubits(*i))
            #sum this new circuit to the existent ones
            self.circ0+=qclarge
    
    #sortFT: add an additional fermionic SWAP circuit to sort the k states. 
    #outputs: 
            # self.circ0: we add the additional circuit to self.circ0 
    def sortFT(self):
        #call the static method circfSWAP for the n qbits
        FSWAPorder=self.circfSWAP.fSWAPSFT(self.numq)
        #add this circuit to self.circ0
        self.circ0+=FSWAPorder
        
        
        
    
    
    #___________________________________________________________________________________________________________________#
    class circFT:
        #Function CH:  Creates the gate CH

        #Parameters: 
                # numq -->  Qbits number of the circuit
                # q0 --> the Qbit wich controls the gate
                # q1 --> the targed Qbit

        #outputs: 
                # qc --> Returns a quantum circuit of nq qbits with the CH gate
        @staticmethod   
        def CH(numq,q0,q1):
            #create a circuit with numq qubits
            qc=Circuit(numq)
            #add the conjugate adjoint of S gate to the qbit q1
            qc.add(gates.S(q1).dagger())
            #add the Hadamard gate to qbit q1
            qc.add(gates.H(q1))
            #add the conjugate adjoint T gate to qbit q1
            qc.add(gates.T(q1).dagger())
            #add the CNOT, control qubit is q0 and the target qbit is q1
            qc.add(gates.CNOT(q0,q1))
            #add the T gate to qbit q1
            qc.add(gates.T(q1))
            #add the Hadamard gate to qbit q1
            qc.add(gates.H(q1))
            #add the S gate to the qbit q1
            qc.add(gates.S(q1))
            #return the circuit with the gate CH
            return qc


        #Function CNOT2:  Creates the gate CNOT2 with is a combination of CNOT surrounded by H gates in each Qbit

        #Parameters: 
                # numq -->  Qbits number of the circuit
                # q0 --> the Qbit wich controls the gate
                # q1 --> the targed Qbit

        #outputs: 
                # qc --> Returns a quantum circuit of nq qbits with the CNOT2 gate
        @staticmethod
        def CNOT2(numq,q0,q1):
            #create a circuit with numq qubits
            qc=Circuit(numq)
            #add the Hadamard gate to qbit q0
            qc.add(gates.H(q0))
            #add the Hadamard gate to qbit q1
            qc.add(gates.H(q1))
            #add the CNOT, control qubit is q0 and the target qbit is q1
            qc.add(gates.CNOT(q0,q1))
            #add the Hadamard gate to qbit q0
            qc.add(gates.H(q0))
            #add the Hadamard gate to qbit q1
            qc.add(gates.H(q1))
            #return the circuit with the gate CNOT2
            return qc



        #F2gate: Gate that implement the 2-Qbit fermionic FT, equivalent to general 2-Qbit fermionic FT with k=0.
        #Created by combination of CZinv.CNOT2.CH.CNOT2; CNOT2 is a control NOT surround by H in each qbit.

        #Function F2gate:  creates the circuit which applies the F2 gate

        #Parameters: 
                # numq -->  Qbits number of the circuit
                # q0,q1 --> the Qbits where the gate is applied

        #outputs: 
                # qc --> Returns a quantum circuit of nq Qbits with the FTW gate
        @staticmethod
        def F2gate(numq,q0,q1):
            #call the @staticmethod CNOT2
            circ0=fermionicFT.circFT.CNOT2(numq,q0,q1)
            #call the @staticmethod CH
            circ1=fermionicFT.circFT.CH(numq,q0,q1)
            #create a circuit with numq qubits
            circ2=Circuit(numq)
            #add the CZ gate, control qubit is q0 and the target qbit is q1
            circ2.add(gates.CZ(q0,q1))
            #sum the circuits with the correct order
            qc=circ0+circ1+circ0+circ2
            #return the circuit with the F2gate
            return qc


        #FTphase: Gate that implement the general phase necessary in the general 2-Qbit fermionic FT 
        #Function FTphase:  creates the circuit which applies the FTW gate

        #Parameters: 
                # numq -->  Qbits number of the circuit
                # q0,q1 --> the Qbits where the gate is applied
                # k --> momentum of the fermions. One of the k values possible of the FT
                # n --> number of points used in the fourier transform.
        #outputs: 
                # qc --> Returns a quantum circuit of nq Qbits with the FTphase gate
        @staticmethod
        def FTphase(numq,q0,q1,k,n):
            #compute the angle for the phi gate
            phi=-2*np.pi*k/float(n)
            #create a circuit with numq qubits
            qc=Circuit(numq)
            # add the Qibo u1 gate in qbit q1 , equivalent to Ph gate
            qc.add(gates.U1(q1, phi))
            #return the circuit with the FTphase
            return qc


        #Ftkgate: Gate that implement the generic 2-Qbit fermionic.
        #Function Ftkgate:  creates the circuit which applies the fFT gate

        #Parameters: 
                # numq -->  Qbits number of the circuit
                # q0,q1 --> the Qbits where the gate is applied
                # k --> momentum of the fermions. One of the k values possible of the FT
                # n --> number of points used in the fourier transform.

        #outputs: 
                # qc --> Returns a quantum circuit of nq Qbits with the fFT gate
        @staticmethod
        def FTkgate(numq,q0,q1,k,n):
            #call the @staticmethod FTphase. It creates a circuit with the FTphase
            circ0=fermionicFT.circFT.FTphase(numq,q0,q1,k,n)
            #call the @staticmethod F2gate. It creates a circuit with the F2gate
            circ1=fermionicFT.circFT.F2gate(numq,q0,q1)
            #sum the circuits, the final one applies first the FTW and then the FT0
            qc=circ0+circ1
            #return the final circuit qcf
            return qc
        
        #Function kvalues:  return the k values used for the FT gate

        #Parameters: 
                # numq --> number of points used in the fourier transform.

        #outputs: 
                # klist --> list with the kvalues
        @staticmethod
        def kvalues(numq):
            n=int(numq/2)
            klist0=np.arange((n)-1,-1,-1)
            klist=klist0[::-1]
            return klist
        
        #Function joinFT:  join the fermionicFT gates in a subcircuit of n-qbits with the corresponding k values
        # from 0 to n/2

        #Parameters: 
                # numq --> number of qbits 

        #outputs: 
                # qc --> return the circuit with the corresponding fFT[n,k]
        @staticmethod
        def joinFT(numq):
            qc=Circuit(numq)
            klist=fermionicFT.circFT.kvalues(numq)
            qbit=0
            for k in klist:
                qc+=fermionicFT.circFT.FTkgate(numq,qbit,qbit+1,k,numq)
                qbit+=2
            
            return qc
            
            
            
        
        
    #__________________________________________________________________________________________________________________
    
    
    class circfSWAP:
        #fSWAPSFT: prepare a circuit with the fSWAP structure needed for the fermionic FT.    
        #Parameters: 
                # numq -->  Qbits number of the circuit

        #outputs: 
                # qc --> Returns a quantum circuit with the fSWAP needed
        @staticmethod
        def fSWAPSFT(numq):
            #create a circuit with numq qubits
            qc=Circuit(numq)
            #create a count0 which start at 1, the fSWAP always start at q1
            count0=1
            #the number of fSWAPS in the first layer is equal to numq/2 -1 and then it decreases by one until there is only
            #1 fSWAP.
            for j in range(int(numq/2.0)-1,0,-1):
                #count1, the first fSWAP is added in the qbit(count0) 
                count1=count0
                for i in range(1,j+1):
                    #add the fSWAP in the circuit
                    qc.add(gates.FSWAP(count1,count1+1))
                    #fSWAP gate is a 2qbit gate, the next gate must be added 
                    count1+=2
                #After the first layer the initial qbit is the ne one
                count0+=1
            return qc
    
   #________________________________________________________________________________________________________________________ 
        
    
    
    

# XY class:

In [4]:
#Class QsimXY1D: creates the qibo quantum circuit that diagonalizes the XY model. Inherits from the class fermionicFT.

    #inputs:
        #Nspin --> number of Qbits targets to perform the XY model simulation. It has to be a power of two 2**m
        #J --> exchange interaction parameter
        #g --> the anisotropy parameter 
        #l --> constant coupling with the magnetic field
        #nqext--> In the case wants to integrate the XY circuit in a larger circuit, the number of Qbits of this larger circuit. 
        #         by default is None, the argument is optional only. However, in the case it is specify, one must then add the 
        #         qlistext input.
        #qlistext--> List where the fermionic FT circuit will be add in the large circuit. The len of this list must be equal as Nspin
    
    #important instances:
            #self.QCXY --> contains the quantum circuit that performs the Jordan Wigner transformation(equivalent to put all X)
            #self.fermFTqc --> contains the quantum circuit that performs the fermionic Fourier Transform (inherit from previous class)
            #self.SWAPBOG --> contains the quantum circuit that sort the final states from fFT to the correct order needed for Bog. transformation
            #self.XYBogqc --> contains the circuit that perform the Bogoulibov transformation
            #self.XY1Dqc --> contains the circuit that transform the spin states to the eigen states of XY 1-D model

    
class QsimXY1D(fermionicFT):
    
    #class QsimXY1D init method: It is called when the class is initialized. Calls all the methods needed to obtain the 
    # XY diagonalizing circuit. The arguments are the ones explained before.
            
    def __init__(self,Nspin,J,g,l,nqext=None,qlistext=None):
        #save the important parameters in a instance
        self.nqXY=Nspin
        self.J=J
        self.g=g
        self.l=l
        #save the number of qubits in a variable nq
        nq=Nspin
        #call the initXqc method
        self.initXqc(nq)
        #call the init method from the class inherited, in this case from fermionicFT
        super().__init__(Nspin)
        #call the BogFSWAP method
        self.BogFSWAP(nq)
        #call the Bogcirc method
        self.Bogcirc(nq)
        
        #save each circuit part in a normal variables
        qcX=self.QCXY
        qcFT=self.fermFTqc
        qcFSWAP=self.SWAPBOG
        qcBog=self.XYBogqc
        
        #sum the circuits in the correct order and save it in a instance
        self.XY1Dqc=qcX+qcFT+qcFSWAP+qcBog
        
        # if the user specify both, nqext and qlistext, then add the FTcircuit on the specified qbits from a large one circuit
        if nqext!=None and qlistext!=None:
            self.exterCirq(nqext,qlistext)
        
        #if the user add a qlistext but none the nqext raise and error
        elif nqext==None and not qlistext==None:
            raise ValueError("Specify both, the number of qbits of the large circuit(nqext)and the list of the qbits you want to use (qlistext)")
        
        ##if the user add the nqext but none the qlistext raise and error
        elif not nqext==None and qlistext==None:
            raise ValueError("Specify both, the number of qbits of the large circuit(nqext)and the list of the qbits you want to use (qlistext)")
        
        #if any external circuit is specified, then pass
        else:
            pass
    
    # initXqc method: create the first part of the circuit, all qbits presents an X gate.
        #arguments:
            #nq--> number of qbits 
        #output:
            #self.QCXY--> instance where the circuit is saved
    
    def initXqc(self,nq):
        #create the quantum circuit with nq qubits
        qc=Circuit(nq)
        #add the Xgates in all Qbits
        qc.add(gates.X(i) for i in range(0,nq))
        #save the circuit in a instance variable
        self.QCXY=qc
    
    # BogFSWAP: create the fermionic SWAP circuit which couples the fermionic FT circuit and the Bogoulibov circuit.
    #           The algorithm consist in create a waterfalls of fermionic SWAPS. The number of fermionic SWAPS waterfalls is
    #           (n/4)-1. Each waterfall is divided in two parts, the down part and the up part. The first down part contains 
    #           numgates(0)=(n/4-1)*4 gates and start at qubit 3, then the follow waterfalls down part contains numgates(i+1)=
    #           =numgates(i)-4 and start in the next qubit from the previous one. For the upper part we only need to follow 
    #           the inverse process explain.
        #arguments:
            #nq--> number of qbits 
        #output:
            #self.SWAPBOG--> instance where the circuit is saved
    def BogFSWAP(self,nq):
        #number of SWAP waterfalls the circuit has
        nfalls=int((nq/4)-1)
        #initialize a quantum circuit with nq qubits
        qcSWAP=Circuit(nq)
        #the first waterfall starts always at the qbit 3
        qbinit=3
        #number of fermionic SWAP of the first waterfall
        numFSWAP=nfalls*4
        #loop to create all the down waterfall parts
        for i in range(0,nfalls):
            #call the static method downfallFSWAP
            qc=self.downfallFSWAP(nq,qbinit,numFSWAP)
            qbinit2=qbinit+numFSWAP-2
            qc2=self.upfallFSWAP(nq,qbinit2,numFSWAP-1)
            #add the downfall circuit into the total circuit
            qcSWAP+=qc+qc2
            #if we are not in the last down fall substract 4 to numFSWAP
            if numFSWAP!=4:
                numFSWAP+=(-4)
            #if we are not in the last down fall sum 1 to the initial qubit 
            if i!=nfalls-1:
                qbinit+=2
        #sum 2 to the initial qubit to now start the up fall
        qbinit+=2
        #The upper falls contains one less gate than the down falls
        numFSWAP=numFSWAP-1
        #loop to create the up falls circuits
        #for j in range(0,nfalls):
            #create the up fall circuit
            #qc=self.upfallFSWAP(nq,qbinit,numFSWAP)
            #add it to the total circuit
            #qcSWAP+=qc
            #sum 4 for the next up fall
            #numFSWAP+=4
            #the initial qbit now increase 3 qbits more
            #qbinit+=+3
        
        #save the total circuit in a new instance
        self.SWAPBOG=qcSWAP.invert()
    
    # BogFSWAP method: create the Bogoulibov transformation circuit. It couples the k and -k modes.
        #arguments:
            #nq--> number of qbits 
        #output:
            #XYBogqc--> instance where the circuit is saved
    def Bogcirc(self,nq):
        #the first k state is always k=0, then it is increasing by one until (n/2)-1
        kcount=0
        #save the physical parameters in variables
        J=self.J
        l=self.l
        g=self.g
        #create the quantum circuit with nq qubits in a instance
        self.XYBogqc=Circuit(nq)
        #Bog gates acts on 2 qubits, loop to add the Bogoulibov gates in all the qubits
        for i in range(0,nq,2):
            #call the staticmethod BogGate, this retuns a circuit with the Bog gate in the qubits specified
            qc=self.BogGate(nq,i,i+1,kcount,nq,J,g,l)
            #increase by one for the following qubit
            kcount+=1
            #sum the qc into the instance
            self.XYBogqc+=qc
    
    # exterCirq method: Introduce the XY simulation circuit into a bigger or equal external circuit in the qubits specified.
        #arguments:
            #nqext--> In the case wants to integrate the XY circuit in a larger circuit, the number of Qbits of this larger circuit. 
            #         by default is None, the argument is optional only. However, in the case it is specify, one must then add the 
            #         qlistext input.
            #qlistext--> List where the fermionic FT circuit will be add in the large circuit. The len of this list must be equal as Nspin 
        #output:
            #XY1Dqc--> instance where the circuit is saved       
    def exterCirq(self,nqext,qlistext):
        largeqc=Circuit(nqext)
        smallqc=self.XY1Dqc
        largeqc.add(smallqc.on_qubits(*qlistext))
        self.XY1Dqc=largeqc
        
    #static method
    #downfallFSWAP: It creates the fermionic SWAP down fall circuit.
        #arguments:
            #nq--> number of qbits 
            #qbinit--> qbit where the fall starts
            #numFSWAP --> number of fermionic SWAPS used
        #output:
            #qc--> return a quantum circuit with the FSWAP down fall
            
    @staticmethod
    def downfallFSWAP(nq,qbinit,numFSWAP):
        #initialize the quantum circuit with nq qubits
        qc=Circuit(nq)
        #save as variable where the circuit starts
        qb0=qbinit
        #loop to add all gates
        for i in range(0,numFSWAP):
            #add the FSWAP gate in the qubits qb0 and qb0+1
            qc.add(gates.FSWAP(qb0,qb0+1))
            #sum 1 to qb0
            qb0+=1
        return qc
    
    #static method
    #upfallFSWAP: It creates the fermionic SWAP down up circuit.
        #arguments:
            #nq--> number of qbits 
            #qbinit--> qbit where the fall starts
            #numFSWAP --> number of fermionic SWAPS used
        #output:
            #qc--> return a quantum circuit with the FSWAP up fall
            
    @staticmethod
    def upfallFSWAP(nq,qbinit,numFSWAP):
        #initialize the quantum circuit with nq qubits
        qc=Circuit(nq)
        #save as variable where the circuit starts
        qb0=qbinit
        for i in range(0,numFSWAP):
            #loop to add all gates
            qc.add(gates.FSWAP(qb0,qb0+1))
            #extract 1 to qb0
            qb0+=-1
        return qc
    
    #static method
    #BogGate: It creates the circuit with the Bogoulibov gate in the qbits specified.
        #arguments:
            #nq--> number of qbits 
            #q0--> first target qbit
            #q1--> second target qbit
            #k --> momentum k state that is coupled in the Bog. transformation
            #n --> the number of qbits variable that appears in the Bog. angle
            #J --> exchange interaction parameter
            #g --> the anisotropy parameter 
            #l --> constant coupling with the magnetic field
        #output:
            #qc--> return a quantum circuit with the Bogoulibov gate in q0 and q1
    
    @staticmethod
    def BogGate(nq,q0,q1,k,n,J,g,l):
        pi=np.pi
        #numerator of the Bogoliubov degree
        num=J*g*np.sin(2.*pi*k/n)
        #denominator of the Bogoliubov degree
        den=l+J*np.cos(2.*pi*k/n)
        #Bogoliubov degree
        theta=np.arctan(num/den)
        #create a circuit of nq qbits
        qc=Circuit(nq)
        #add the CNOT gate, where the q0 is the control qbit and q1 the target one
        qc.add(gates.CNOT(q0,q1))
        #add the X gates in q1
        qc.add(gates.X(q1))
        #add control RX gate, where the control is the q1 and the the target is q0. The minus due to the Qibo CRX have the sinus
        #components with a -, so we need to correct it
        qc.add(gates.CRX(q1,q0, -theta))
        #add the X gates in q1
        qc.add(gates.X(q1))
        #add the CNOT gate, where the q0 is the control qbit and q1 the target one
        qc.add(gates.CNOT(q0,q1))
        #return the circuit with the bogoliubov gate
        return qc
        
        
            
        
        
        